In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
import os

In [4]:
# Set the path to the directory containing the colored and natural coral reef images
#data_dir = r'/kaggle/input/healthy-and-bleached-corals-image-classification'

#import zipfile
#with zipfile.ZipFile("/work/data/archive (1).zip") as zip_ref:
#    zip_ref.extractall("data")

KeyboardInterrupt: 

In [2]:
data_dir = "../src/data/"

In [3]:
# Define image dimensions, batch size, and train/test split ratio
img_width, img_height = 150, 150
batch_size = 32
validation_split = 0.2

In [4]:
# Create data generator with data augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=validation_split
)

# Load and split the data into train and test sets
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 739 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [5]:
# Create a CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [21]:
def is_image_corrupted(image_path):
    try:
        # Open the image file
        Image.open(image_path).verify()
        return False  # Image is not corrupted
    except (IOError, SyntaxError):
        return True  # Image is corrupted

def delete_corrupted_images(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if is_image_corrupted(file_path):
            print(f"Deleting corrupted image: {filename}")
            os.remove(file_path)

# Set the path to the directory containing the images
image_directory = '../src/data/bleached_corals/'
image_directory2 = '../src/data/healthy_corals/'


# Delete corrupted images
delete_corrupted_images(image_directory)
delete_corrupted_images(image_directory2)

In [22]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10

In [23]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

/tmp/ipykernel_1029/3723281591.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [13]:
# Evaluate the model on the test set
_, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.